#Introdução à inteligência artificial

##Projeto final - Tumor cerebral 

Esse tema de projeto é considerado como desafio, devido à dificuldade de tratamento de dados e devido a arquitetura da rede neural. Pois será utilizada camadas convolucionais que processam imagens de ressonância magnética e detectam a localização do tumor.

O dataset escolhido foi: https://figshare.com/articles/brain_tumor_dataset/1512427

##Download do dataset

In [ ]:
!mkdir brain_tumor
!wget https://ndownloader.figshare.com/articles/1512427/versions/5
!unzip 5
!unzip brainTumorDataPublic_1-766.zip -d ./brain_tumor
!unzip brainTumorDataPublic_1533-2298.zip -d ./brain_tumor
!unzip brainTumorDataPublic_767-1532.zip  -d ./brain_tumor
!unzip brainTumorDataPublic_2299-3064.zip  -d ./brain_tumor
!rm *.zip

##Importação das bibliotecas

In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import h5py

##Leitura e tratamento das informações

Vamos ler somente o primeiro arquivo para verificar qual seu conteúdo

In [ ]:
data_path = "/content/brain_tumor/"
with h5py.File(data_path+'1.mat', 'r') as f:
  print(f.keys())

O arquivo contém várias informações: O ID do paciente, a imagem de ressonancia, o label, as coordenada do tumor e a a máscara binaria da localização do tumor


In [ ]:
with h5py.File(data_path+'1.mat', 'r') as f:
  print(f['cjdata'].keys())

Vamos plotar as imagens de interesse: imagem do exame e a máscara binária.
A máscara indica a posição exata do tumor na imagem do exame, ou seja, se fizermos uma operação AND entre a máscara binária e a imagem do exame teremos como resultado uma imagem somente com o tumor.

In [ ]:
from numpy import savetxt
with h5py.File(data_path+'1.mat', 'r') as f:
  plt.imshow(f['cjdata']['tumorMask'],cmap='gray')
  plt.figure()
  plt.imshow(f['cjdata']['image'],cmap='gray')

Em seguida, serão lidos todos os arquivos que compoẽm o _dataset_. Como poucas imagens são de tamanhos diferentes, foi decidido retirar imagens diferente de 512x512 pixels.
As imagens dos exames são de 11 bits! Portanto, precisamos mudar a escala de 0 a 2047 para uma escala de 0 a 1.

Em seguida calculamos a média e desvio padrão do dataset para fazer a normalização.

In [ ]:
data=[]
target=[]
test = 0
for file_number in range(1,3065):
  with h5py.File(data_path+'{}.mat'.format(file_number), 'r') as f:
    if(list(torch.from_numpy(f['cjdata']['image'][:]).shape) == [512,512]):
      if(list(torch.from_numpy(f['cjdata']['tumorMask'][:]).shape) == [512,512]):
        data.append(np.asarray(f['cjdata']['image'][:]))
        target.append(np.asarray(f['cjdata']['tumorMask'][:]))

tensor_data = torch.tensor(np.asarray(data))
tensor_target = torch.tensor(np.asarray(target))

mean = 0.
std = 0.
tensor_data = tensor_data.type(torch.FloatTensor)
tensor_data = tensor_data/2047
for images in tensor_data:
    mean += torch.mean(images)
    std += torch.std(images)

mean /= len(tensor_data)
std /= len(tensor_data)

image_dataset = torch.utils.data.TensorDataset(tensor_data,tensor_target) 

Classe auxiliar para aplicar transformadas em cada dataset separado

In [ ]:
class ApplyTransform(Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

    def __len__(self):
        return len(self.dataset)

##Definição do tamanho de lote e transformações de cada _subset_ do _dataset_

In [ ]:
# Porcentagem da quantidade de imagem que será definida para uso como validação e teste
# TODO: Defina o valor percentual que esteja entre 0 a 1
test_valid_size = 0.4

In [ ]:
# TODO: Quantas amostras por lote 
batch_size = 4

# Define as transformações, fique a vontade para adicionar mais transformações!!!
train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Normalize(mean, std)])
valid_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Normalize(mean, std)])
test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Normalize(mean, std)])
target_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.ToTensor()])
# Obter índices de treinamento que serão usados para validação
num_train = len(image_dataset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(test_valid_size * num_train))
train_idx, valid_idx , test_idx = indices[split:], indices[:int(np.floor(split/2))], indices[int(np.floor(split/2)):split]

# Definir amostradores para obter lotes de treinamento e validação
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

# Aplica as transformações para o treino e validação
train_dataset = ApplyTransform(image_dataset, transform=train_transform, target_transform=target_transform)
valid_dataset = ApplyTransform(image_dataset, transform=valid_transform,target_transform=target_transform)
test_dataset = ApplyTransform(image_dataset, transform=test_transform,target_transform=target_transform)

# Preparar carregadores de dados (combinar conjunto de dados e amostrador)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
    sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, 
    sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
    sampler=test_sampler)

data_loader = {'train':train_loader,'val':valid_loader,'test':test_loader}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##Definição e treinamento do modelo

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Época {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Cada época tem uma fase de treino e validação
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Modelo em treinamento
            else:
                model.eval()   # Modelo em avaliação

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zera o gradiente do otimizador
                optimizer.zero_grad()

                # Analisa somente as perdas se for no treinamento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    #y_pred = outputs.data.cpu().numpy().ravel()
                    #y_true = labels.data.cpu().numpy().ravel()

                    # 'loss.backward()' + 'optimizer.step()' somente no treinamento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estatisticas
                running_loss += loss.item() * inputs.size(0)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss 

            print('{} Perda: {:.4f}'.format(
                phase, epoch_loss))

            # Copia o modelo
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Treinamento completo em {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    torch.save(best_model_wts,"model.pt")

    # Carrega os pesos do melhor modelo
    model.load_state_dict(best_model_wts)
    return model

Para a definição de uma rede neural para segmentação, você pode usar as seguintes redes pré-treinadas: [U-Net](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/), [DeepLabv3](https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/) ou [FCN](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/). Como você tambem pode criar uma do zero e aproveitando partes pré-treinadas de outras redes como a [VGG](https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.vgg11), [Resnet](https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.resnet18) e etc.

In [ ]:
# TODO: Defina aqui qual seu modelo
model = NotImplemented

for param in model.parameters():
    param.requires_grad = False
                      
# Move o modelo para o dispositivo disponivel
model = model.to(device)

criterion = NotImplemented
optimizer = NotImplemented
exp_lr_scheduler = NotImplemented

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=30)

##Teste o modelo

In [ ]:
# Carrega o modelo 
file_name = "model.pt"
model.load_state_dict(torch.load(file_name))

Vamos ler uma __imagem__ e __label__ do dataset de teste por vez e plotar para visualizar. Depois faremos a inferência dessa imagem e visualizaremos o resultado

In [ ]:
data_iter = iter(test_loader)
image, label = data_iter.next()

In [ ]:
#Visualização da máscara binária
plt.imshow(label[0][0],cmap="gray")

In [ ]:
#Visualização da imagem do exame
plt.imshow(image[0][0],cmap="gray")

In [ ]:
#Relizar a inferência
model.eval()
out = model(image.cuda())

In [ ]:
#Visualizar o resultado
output = (out[0][0]).cpu().detach().numpy()
plt.imshow(np.asarray(output),cmap="gray")